In [2]:
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/germa/thesis/bachelor_project2023/src/utils')
from chunker import Chunker
import torch.nn as nn
import torch
from torch import Tensor
from typing import List, Tuple
from torch.utils.data import TensorDataset, DataLoader, random_split

/Users/germa/anaconda3/envs/project/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/c6/k78nzngn7nbdd0786bxl3s0c0000gn/T/ipykernel_3478/1105922383.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [17]:
class XLMRoberta(nn.Module):
    def __init__(self, model_name: str):
        super(XLMRoberta, self).__init__()
        self.model = XLMRobertaModel.from_pretrained(model_name)
        self.tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
        self.chunker = Chunker(self.tokenizer, 512)
        self.train_df = self.get_data('../../../data/train/train.csv')
    
    def get_data(self, path):
        train_df = pd.read_csv(path)
        train_short_df = train_df.head(5)
        return train_short_df
    
    def chunk_data(self, df):
        texts1, texts2 = df["text1"], df["text2"]
        input_ids = []
        for i in range(len(texts1)):
            input_id_1 = self.chunker.chunk(texts1[i])
            input_id_2 = self.chunker.chunk(texts2[i])
            input_ids.append([input_id_1, input_id_2])           
        return input_ids
    
    def run(self):
        input_ids = self.chunk_data(self.train_df)
        output = []
        for i in range(len(input_ids)):
            row_emb = []
            for text in input_ids[i]:
                chunks_emb = []
                for chunk in text:
                    chunk_ids = torch.tensor(chunk).unsqueeze(0)
                    attention_mask = [1 if i != 0 else 0 for i in chunk_ids.tolist()]
                    attention_mask = torch.tensor(attention_mask).unsqueeze(0)
                    outputs= self.model(chunk_ids, attention_mask)
                    last_hidden_state = outputs.last_hidden_state.mean(dim=1)
                    chunks_emb.append(last_hidden_state)
                row_emb.append(chunks_emb)
            output.append(row_emb)
        return output
    
        

In [18]:
rob = XLMRoberta("xlm-roberta-base")


In [19]:
rob.run()

Token indices sequence length is longer than the specified maximum sequence length for this model (748 > 512). Running this sequence through the model will result in indexing errors


[[[tensor([[ 2.1009e-02,  4.8251e-02, -6.7564e-02,  2.2767e-02,  1.1471e-02,
            -1.4763e-02, -6.8214e-02,  8.6669e-02, -6.9803e-03, -6.6909e-02,
             3.1757e-02,  1.9798e-02, -2.2936e-02,  9.2840e-03, -9.3987e-02,
             1.6521e-02,  3.3346e-02, -3.4174e-02,  1.0650e-01,  2.5400e-02,
             5.4198e-02,  1.7682e-02,  3.5283e-02,  1.1058e-01,  6.5043e-02,
            -7.6108e-02,  1.4128e-01,  8.1868e-02,  8.5008e-02, -3.1740e-02,
             1.2435e-01, -2.3330e-02,  5.5516e-03,  6.7962e-04,  5.3947e-02,
             5.4963e-02,  6.0809e-02, -5.0919e-02,  7.3754e-02,  7.0148e-02,
             1.1670e-01,  1.3618e-02,  2.6382e-01,  2.4767e-02, -6.1342e-03,
            -5.8086e-02,  2.8411e-02,  1.0294e-02, -8.6277e-02, -6.3398e-02,
             1.8579e-02, -5.2039e-03,  1.6510e-01,  6.6333e-02, -1.7553e-01,
            -1.2006e-01,  5.7579e-02, -1.2334e-02, -6.6156e-03,  6.6941e-02,
            -5.2942e-02,  1.0557e-01, -1.3126e-02,  4.0118e-02,  6.1581e-02,